In [4]:
from collections import Counter
import json
from copy import deepcopy
import numpy as np

In [5]:
# Task 1: Vocabulary Creation (20 points)
def create_vocabulary():
    # Read the file
    # Count the occurrence of words in a temporary dictionary 'word_counter' first
    with open('data/train') as file:
        train_data = []
        temp = []
        word_counter = Counter()
        s = file.read().splitlines()
        for i in range(len(s)):
            if s[i] == '':
                train_data.append(temp)
                temp = []
                continue
            one_line = s[i].split('\t')
            word_counter[one_line[1]] += 1
            temp.append(one_line)
        train_data.append(temp)

    # Set a minimum threshold 'threshold'
    # If a word occurs less than 'threshold' times, remap it as '<unk>' in the final dictionary 'word_counter_final'
    threshold = 2
    word_counter_final = Counter()
    for k, v in word_counter.items():
        if v <= threshold:
            word_counter_final['<unk>'] += v
        else:
            word_counter_final[k] = v

    # Write the dictionary to vocab.txt
    with open('vocab.txt', 'w') as file:ﬁ
        file.write(f"<unk>\t0\t{word_counter_final['<unk>']}\n")
        word_counter_final.pop('<unk>', None)
        i = 1
        for word, freq in word_counter_final.most_common():
            s = f"{word}\t{i}\t{freq}\n"
            i += 1
            file.write(s)
    return word_counter_final, train_data

In [6]:
word_counter_final, train_data = create_vocabulary()

In [73]:
word_counter_final.most_common()

[(',', 46476),
 ('the', 39533),
 ('.', 37452),
 ('of', 22104),
 ('to', 21305),
 ('a', 18469),
 ('and', 15346),
 ('in', 14609),
 ("'s", 8872),
 ('for', 7743),
 ('that', 7723),
 ('$', 6762),
 ('is', 6735),
 ('``', 6673),
 ('The', 6578),
 ("''", 6500),
 ('said', 5418),
 ('on', 4905),
 ('%', 4718),
 ('it', 4509),
 ('by', 4274),
 ('from', 4238),
 ('at', 4142),
 ('million', 4122),
 ('as', 4054),
 ('with', 3987),
 ('Mr.', 3856),
 ('are', 3629),
 ('was', 3615),
 ('be', 3584),
 ('its', 3382),
 ('has', 3184),
 ("n't", 3132),
 ('an', 3015),
 ('have', 2993),
 ('will', 2982),
 ('he', 2496),
 ('company', 2429),
 ('or', 2429),
 ('which', 2164),
 ('year', 2162),
 ('would', 2108),
 ('--', 2005),
 ('about', 1991),
 ('says', 1948),
 ('they', 1904),
 ('this', 1844),
 ('more', 1842),
 ('were', 1776),
 ('market', 1760),
 ('In', 1706),
 ('billion', 1680),
 ('But', 1668),
 ('their', 1655),
 ('up', 1625),
 ('had', 1625),
 ('than', 1586),
 ('U.S.', 1552),
 ('but', 1552),
 ('his', 1547),
 ('who', 1509),
 ('been'

In [74]:
len(word_counter_final)

16919

In [8]:
# Task 2: Model Learning (20 points)
def model_learning(word_counter_final, data):
    #Get the emission and transition count first
    #Maintain a tag_counter that counts the number of occurrences of each tag
    tag_counter = Counter()
    emission_probabilities, transition_probabilities = Counter(), Counter()
    for i, sentence in enumerate(data):
        for j, word_desc in enumerate(sentence):
            #If word is not in word_counter_final, we handle it as <unk> special token.
            if word_desc[1] not in word_counter_final:
                word_desc[1] = '<unk>'
            tag_counter[word_desc[2]] += 1
            emission_probabilities[(word_desc[2], word_desc[1])] += 1
            #For the first word, use <start> as the previous token.
            if j == 0:
                transition_probabilities[('<start>', word_desc[2])] += 1
            #For the last word, we don't have to calculate transition_probabilities.
            elif j == len(sentence):
                continue
            else:
                transition_probabilities[(sentence[j - 1][2], sentence[j][2])] += 1

    tag_counter['<start>'] = len(data)

    #Use the tag_counter to normalize emission_probabilities and transition_probabilities
    for key, val in emission_probabilities.items():
        emission_probabilities[key] = val / tag_counter[key[0]]
    for key, val in transition_probabilities.items():
        transition_probabilities[key] = val / tag_counter[key[0]]

    # Write transition and emission probabilities to hmm.json
    js = {}
    t = {}
    e = {}
    for k, v in transition_probabilities.items():
        t[repr(k)] = v
    for k, v in emission_probabilities.items():
        e[repr(k)] = v
    js['transition'] = t
    js['emission'] = e
    with open("hmm.json", "w") as outfile:
        json.dump(js, outfile)

    return tag_counter, emission_probabilities, transition_probabilities

In [9]:
tag_counter, emission_probabilities, transition_probabilities = model_learning(word_counter_final, train_data)

In [10]:
tag_counter

Counter({'NNP': 87608,
         ',': 46480,
         'CD': 34876,
         'NNS': 57859,
         'JJ': 58944,
         'MD': 9437,
         'VB': 25489,
         'DT': 78775,
         'NN': 127534,
         'IN': 94758,
         '.': 37883,
         'VBZ': 20982,
         'VBG': 14348,
         'CC': 22817,
         'VBD': 28309,
         'VBN': 19330,
         'RB': 29621,
         'TO': 21461,
         'PRP': 16766,
         'RBR': 1675,
         'WDT': 4194,
         'VBP': 12326,
         'RP': 2515,
         'PRP$': 7989,
         'JJS': 1867,
         'POS': 8284,
         '``': 6782,
         'EX': 833,
         "''": 6622,
         'WP': 2285,
         ':': 4680,
         'JJR': 3174,
         'WRB': 2050,
         '$': 6937,
         'NNPS': 2505,
         'WP$': 166,
         '-LRB-': 1305,
         '-RRB-': 1321,
         'PDT': 333,
         'RBS': 435,
         'FW': 224,
         'UH': 87,
         'SYM': 55,
         'LS': 47,
         '#': 127,
         '<start>': 3821

In [11]:
emission_probabilities

Counter({('NNP', 'Pierre'): 6.84868961738654e-05,
         ('NNP', '<unk>'): 0.09307369190028308,
         (',', ','): 0.9999139414802065,
         ('CD', '61'): 0.0007168253240050465,
         ('NNS', 'years'): 0.019530237301024905,
         ('JJ', 'old'): 0.003613599348534202,
         ('MD', 'will'): 0.3138709335593939,
         ('VB', 'join'): 0.0015693044058221193,
         ('DT', 'the'): 0.5016439225642653,
         ('NN', 'board'): 0.0023287907538381922,
         ('IN', 'as'): 0.0353954283543342,
         ('DT', 'a'): 0.2341478895588702,
         ('JJ', 'nonexecutive'): 0.00010179153094462541,
         ('NN', 'director'): 0.002422883309548826,
         ('NNP', 'Nov.'): 0.0026709889507807506,
         ('CD', '29'): 0.0021218029590549374,
         ('.', '.'): 0.9886228651373967,
         ('NNP', 'Mr.'): 0.044014245274404167,
         ('VBZ', 'is'): 0.3208940997045086,
         ('NN', 'chairman'): 0.0033638088666551663,
         ('IN', 'of'): 0.23322569070685326,
         ('NNP', '

In [12]:
transition_probabilities

Counter({('<start>', 'NNP'): 0.19789104610393007,
         ('NNP', 'NNP'): 0.3782645420509543,
         ('NNP', ','): 0.13846908958086018,
         (',', 'CD'): 0.021234939759036144,
         ('CD', 'NNS'): 0.15775891730703062,
         ('NNS', 'JJ'): 0.017196978862406887,
         ('JJ', ','): 0.029129343105320303,
         (',', 'MD'): 0.010542168674698794,
         ('MD', 'VB'): 0.7990886934407121,
         ('VB', 'DT'): 0.22209580603397544,
         ('DT', 'NN'): 0.4734877816566169,
         ('NN', 'IN'): 0.24741637524111218,
         ('IN', 'DT'): 0.32807784039342325,
         ('DT', 'JJ'): 0.21834338305299905,
         ('JJ', 'NN'): 0.4491042345276873,
         ('NN', 'NNP'): 0.009519030219392476,
         ('NNP', 'CD'): 0.019176330928682313,
         ('CD', '.'): 0.0725427227893107,
         ('NNP', 'VBZ'): 0.0391973335768423,
         ('VBZ', 'NN'): 0.035792584119721665,
         ('IN', 'NNP'): 0.14870512252263662,
         (',', 'DT'): 0.1336273666092943,
         ('DT', 'NNP'

In [13]:
with open('data/dev') as file:
    s = file.read().splitlines()
    dev_data = []
    temp = []
    ground_truth_tags = []
    for d in s:
        if d != '':
            word_desc = d.split('\t')
            temp.append(word_desc)
            ground_truth_tags.append(word_desc[2])
        else:
            dev_data.append(temp)
            temp = []
    dev_data.append(temp)

In [17]:
def accuracy(ground_truth_tags, predicted_tags):
    return sum(
        [a == b for a, b in zip(ground_truth_tags, [word for sublist in predicted_tags for word in sublist])]) / len(
        ground_truth_tags)

In [75]:
# Task 3: Greedy Decoding with HMM (30 points)
def greedy_decoding(data, word_counter_final, tag_counter, emission_probabilities, transition_probabilities):
    #Keep track of predicted tags for entire document in predicted_tags_greedy
    predicted_tags_greedy = []
    unique_tags = [x for x in tag_counter.keys() if x != "<start>"]
    for sentence in deepcopy(data):
        #Keep track of predicted tags for one sentence in word_tags
        word_tags = []
        #In the beginning start with a previous_tag of <start>. Required to calculate transition_probabilities
        prev_tag = '<start>'
        for word_desc in sentence:
            s = -1
            #If word is not in word_counter_final, we handle it as <unk> special token
            if word_desc[1] not in word_counter_final:
                word_desc[1] = '<unk>'
            #For each word, we try all possible tags and calculate the ep*tp
            #We assign the tag that gives maximum ep*tp to the word and proceed
            for tag in unique_tags:
                ep = emission_probabilities.get((tag, word_desc[1]), 0)
                tp = transition_probabilities.get((prev_tag, tag), 0)

                if (ep * tp) > s:
                    s = ep * tp
                    temp_tag = tag

            prev_tag = temp_tag
            word_tags.append(temp_tag)
        #         print(word_tags)
        predicted_tags_greedy.append(word_tags)
    return predicted_tags_greedy


In [76]:
dev_greedy_tags = greedy_decoding(dev_data, word_counter_final, tag_counter, emission_probabilities,
                                      transition_probabilities)
print(f"Accuracy dev_data with Greedy Decoding = {accuracy(ground_truth_tags, dev_greedy_tags)}")

Accuracy dev_data with Greedy Decoding = 0.9298615748891992


In [77]:
#Task 4: Viterbi Decoding with HMM (30 Points)
def viterbi_decoding(data, word_counter_final, tag_counter, emission_count, transition_count):
    unique_tags = [x for x in tag_counter.keys() if x != "<start>"]

    def viterbi_one_sentence(sentence, unique_tags):
        n = len(sentence)
        sent = [elem[1] for elem in sentence]
        #Initialize viterbi_matrix which has viterbi values for possible tags for words of a sentence 
        viterbi_matrix = np.zeros((len(unique_tags), len(sent)))
        #Initialize backpointer_matrix which keeps track of tag that gives maximum viterbi values
        backpointer_matrix = np.zeros((len(unique_tags), len(sent))).astype(int)
        for i, tag in enumerate(unique_tags):
            #Use the initial probability distribution to set the first column of viterbi_matrix
            first_word = sent[0] if sent[0] in word_counter_final else '<unk>'
            viterbi_matrix[i][0] = transition_count.get(('<start>', tag), 0) * emission_count.get((tag, first_word),0)
        for i, word in enumerate(sent[1:]):
            for j, tag in enumerate(unique_tags):
                #Handle unknown words as <unk>
                if word not in word_counter_final:
                    word = '<unk>'
                probs = [(viterbi_matrix[x][i] * transition_count.get((unique_tags[x], tag),
                                                                      0) * emission_count.get((tag, word), 0)) for x
                         in range(len(unique_tags))]
                
                #The value of each cell of viterbi_matrix is computed by  
                #recursively taking the most probable path that could lead us to this tag.
                viterbi_matrix[j][i + 1] = max(probs)
                #Store the index to this path in backpointer matrix 
                backpointer_matrix[j][i + 1] = np.argmax(np.array(probs))

        #Find the most likely tag of the last word using the last column of viterbi_matrix
        #Follow the backpointer_matrix corresponding to this tag to decode the predicted tags
        best_path_pointer = np.argmax(viterbi_matrix[:, n - 1])
        tags_for_the_sentence = []
        best_decoded_path = []
        for k in range(n - 1, -1, -1):
            best_decoded_path.append(best_path_pointer)
            best_path_pointer = backpointer_matrix[best_path_pointer][k]
        best_decoded_path = reversed(best_decoded_path)
        for tag_ind in best_decoded_path:
            tags_for_the_sentence.append(unique_tags[tag_ind])
        return tags_for_the_sentence
    
    #Keep track of predicted tags for the entire document in predicted_tags_greedy
    predicted_tags_viterbi = []
    for sentence in data:
        #For each sentence run viterbi_one_sentence which returns predicted tags for a sentence
        predicted_tags_viterbi.append(viterbi_one_sentence(sentence, unique_tags))

    return predicted_tags_viterbi


In [78]:
dev_viterbi_tags = viterbi_decoding(dev_data, word_counter_final, tag_counter, emission_probabilities,
                                        transition_probabilities)
print(f"Accuracy dev_data with Viterbi Decoding = {accuracy(ground_truth_tags, dev_viterbi_tags)}")

Accuracy dev_data with Viterbi Decoding = 0.9436813186813187


In [70]:
def heuristic_learn(train_data):
    unambiguous_tags = dict()
    for i, sent in enumerate(train_data):
        for j, word in enumerate(sent):
            if word[1] in unambiguous_tags and unambiguous_tags[word[1]] != word[2]:
                unambiguous_tags[word[1]] = '<ambiguous>'
                continue
            unambiguous_tags[word[1]] = word[2]
    return unambiguous_tags

def heuristic_apply(dev_data, unambiguous_tags, predicted_tags):
    corrected_count = 0
    for i, sent in enumerate(dev_data):
        for j, word in enumerate(sent):
            if word[1] in unambiguous_tags and unambiguous_tags[word[1]] != '<ambiguous>':
                if unambiguous_tags[word[1]] != predicted_tags[i][j]:
                    corrected_count += 1
                    print(f"For word {word} HMM predicted tag: {predicted_tags[i][j]}, Heuristic tag: {unambiguous_tags[word[1]]}")
                    predicted_tags[i][j] = unambiguous_tags[word[1]]
                    
    return corrected_count

In [71]:
# Print With heuristic
unambiguous_tags = heuristic_learn(train_data)

greedy_corrected_count = heuristic_apply(dev_data, unambiguous_tags, dev_greedy_tags)
print(f"\nAccuracy dev_data with Greedy Decoding and Heuristic = {accuracy(ground_truth_tags, dev_greedy_tags)}")
print(f"Heuristic corrected {greedy_corrected_count} in Greedy output.\n")

viterbi_corrected_count = heuristic_apply(dev_data, unambiguous_tags, dev_viterbi_tags)
print(f"\nAccuracy dev_data with Viterbi Decoding and Heuristic = {accuracy(ground_truth_tags, dev_viterbi_tags)}")
print(f"Heuristic corrected {viterbi_corrected_count} in Heuristic output.")

For word ['7', '--', ':'] HMM predicted tag: NNP, Heuristic tag: :
For word ['24', 'their', 'PRP$'] HMM predicted tag: NNP, Heuristic tag: PRP$
For word ['5', 'camera', 'NN'] HMM predicted tag: NNP, Heuristic tag: NN
For word ['6', '...', ':'] HMM predicted tag: NNP, Heuristic tag: :
For word ['16', '$', '$'] HMM predicted tag: NNP, Heuristic tag: $
For word ['42', '``', '``'] HMM predicted tag: NNP, Heuristic tag: ``
For word ['11', ',', ','] HMM predicted tag: NNP, Heuristic tag: ,
For word ['13', ')', '-RRB-'] HMM predicted tag: NNP, Heuristic tag: -RRB-
For word ['13', 'customer', 'NN'] HMM predicted tag: NNP, Heuristic tag: NN
For word ['22', 'again', 'RB'] HMM predicted tag: NNP, Heuristic tag: RB
For word ['5', 'or', 'CC'] HMM predicted tag: NNP, Heuristic tag: CC
For word ['16', 'attention', 'NN'] HMM predicted tag: NNP, Heuristic tag: NN
For word ['7', '.', '.'] HMM predicted tag: NNP, Heuristic tag: .
For word ['31', '.', '.'] HMM predicted tag: NNP, Heuristic tag: .
For word

In [79]:
unambiguous_tags

{'Pierre': 'NNP',
 '<unk>': '<ambiguous>',
 ',': ',',
 '61': 'CD',
 'years': 'NNS',
 'old': 'JJ',
 'will': '<ambiguous>',
 'join': '<ambiguous>',
 'the': '<ambiguous>',
 'board': 'NN',
 'as': '<ambiguous>',
 'a': '<ambiguous>',
 'nonexecutive': 'JJ',
 'director': 'NN',
 'Nov.': '<ambiguous>',
 '29': 'CD',
 '.': '.',
 'Mr.': 'NNP',
 'is': '<ambiguous>',
 'chairman': 'NN',
 'of': '<ambiguous>',
 'N.V.': 'NNP',
 'Dutch': '<ambiguous>',
 'publishing': '<ambiguous>',
 'group': '<ambiguous>',
 'Rudolph': 'NNP',
 'Agnew': 'NNP',
 '55': 'CD',
 'and': '<ambiguous>',
 'former': '<ambiguous>',
 'Consolidated': 'NNP',
 'Gold': '<ambiguous>',
 'Fields': '<ambiguous>',
 'PLC': 'NNP',
 'was': 'VBD',
 'named': '<ambiguous>',
 'this': '<ambiguous>',
 'British': '<ambiguous>',
 'industrial': 'JJ',
 'conglomerate': '<ambiguous>',
 'A': '<ambiguous>',
 'form': '<ambiguous>',
 'asbestos': 'NN',
 'once': '<ambiguous>',
 'used': '<ambiguous>',
 'to': '<ambiguous>',
 'make': '<ambiguous>',
 'Kent': 'NNP',
 'c

In [82]:
for k, v in unambiguous_tags.items():
    if unambiguous_tags[k] != '<ambiguous>':
        print(k,v)

Pierre NNP
, ,
61 CD
years NNS
old JJ
board NN
nonexecutive JJ
director NN
29 CD
. .
Mr. NNP
chairman NN
N.V. NNP
Rudolph NNP
Agnew NNP
55 CD
Consolidated NNP
PLC NNP
was VBD
industrial JJ
asbestos NN
Kent NNP
cigarette NN
percentage NN
cancer NN
deaths NNS
among IN
workers NNS
it PRP
30 CD
researchers NNS
fiber NN
crocidolite NN
unusually RB
resilient JJ
enters VBZ
lungs NNS
exposures NNS
causing VBG
symptoms NNS
decades NNS
unit NN
Loews NNP
cigarettes NNS
using VBG
its PRP$
1956 CD
Although IN
preliminary JJ
findings NNS
were VBD
year NN
today NN
England NNP
forum NN
new JJ
attention NN
problem NN
`` ``
story NN
We PRP
're VBP
talking VBG
anyone NN
having VBG
questionable JJ
properties NNS
our PRP$
products NNS
now RB
'' ''
nor CC
who WP
aware JJ
smokers NNS
useful JJ
information NN
users NNS
James NNP
Talcott NNP
Boston NNP
Institute NNP
Dr. NNP
from IN
medical JJ
schools NNS
Harvard NNP
spokeswoman NN
modest JJ
paper NN
different JJ
type NN
From IN
1953 CD
1955 CD
9.8 CD
billion C

In [72]:
with open('data/test') as file:
    s = file.read().splitlines()
    test_data = []
    temp = []
    for d in s:
        if d != '':
            word_desc = d.split('\t')
            temp.append(word_desc)
        else:
            test_data.append(temp)
            temp = []
    test_data.append(temp)
# print(test_data)
test_greedy_tags = greedy_decoding(test_data, word_counter_final, tag_counter, emission_probabilities,
                                   transition_probabilities)

heuristic_apply(test_data, unambiguous_tags, test_greedy_tags)

with open('greedy.out', 'w') as file:
    for i, sentence in enumerate(test_data):
        for j, word_desc in enumerate(sentence):
            write = f"{j + 1}\t{word_desc[1]}\t{test_greedy_tags[i][j]}\n"
            file.write(write)
        file.write('\n')

test_viterbi_tags = viterbi_decoding(test_data, word_counter_final, tag_counter, emission_probabilities,
                                     transition_probabilities)

heuristic_apply(test_data, unambiguous_tags, test_viterbi_tags)
#
with open('viterbi.out', 'w') as file:
    for i, sentence in enumerate(test_data):
        for j, word_desc in enumerate(sentence):
            write = f"{j + 1}\t{word_desc[1]}\t{test_viterbi_tags[i][j]}\n"
            file.write(write)
        file.write('\n')